In [3]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import numpy as np

In [437]:
X,Y=make_moons(n_samples=500,noise=0.30,random_state=42)
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=42)

voting=VotingClassifier(
    estimators=[
        ('lr',LogisticRegression(random_state=42)),
        ('rf',RandomForestClassifier(random_state=42)),
        ('svc',SVC(random_state=42))
    ]
)
voting.fit(x_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [253]:
for name, clf in voting.named_estimators_.items():
    print(name,"=", clf.score(x_test,y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [255]:
[clf.predict(x_test[[0]]) for clf in voting.estimators_]

[array([1]), array([1]), array([0])]

In [257]:
voting.score(x_test,y_test)

0.912

In [259]:
voting.voting='soft'
voting.named_estimators['svc'].probability=True
voting.fit(x_train,y_train)
voting.score(x_test,y_test)

0.92

In [295]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagging=BaggingClassifier(DecisionTreeClassifier(),oob_score=True,n_estimators=500,max_samples=100,n_jobs=-1)
bagging.fit(x_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, oob_score=True)

In [301]:
bagging.oob_score_

0.9173333333333333

In [293]:
tree=DecisionTreeClassifier().fit(x_train,y_train)
tree.score(x_test,y_test)

0.856

In [309]:
bagging.score(x_test,y_test)

0.92

In [313]:
bagging.oob_decision_function_[:3]

array([[0.33243243, 0.66756757],
       [0.44186047, 0.55813953],
       [1.        , 0.        ]])

In [327]:
fores=RandomForestClassifier(n_estimators=500,n_jobs=-1,max_leaf_nodes=16,random_state=42).fit(x_train,y_train)
fores.score(x_test,y_test)

0.912

In [351]:
bagging_sq=BaggingClassifier(DecisionTreeClassifier(max_features='sqrt',max_leaf_nodes=16,splitter="random"),n_estimators=500,n_jobs=-1,random_state=42)
bagging_sq.fit(x_train,y_train)
bagging_sq.score(x_test,y_test)

0.912

In [337]:
np.all(fores.predict(x_test)==bagging_sq.predict(x_test))

True

In [353]:
fores.feature_importances_

array([0.42253629, 0.57746371])

In [405]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

ada=AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=30,random_state=42,learning_rate=0.3)
np.mean(cross_val_score(ada,x_train,y_train,cv=5))

0.9066666666666666

In [431]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier()

GradientBoostingClassifier()

In [482]:
from sklearn.ensemble import StackingClassifier

stacking_clf=StackingClassifier(
    estimators=[
    ('lr',LogisticRegression(random_state=42)),
    ('rf',RandomForestClassifier(random_state=42)),
    ('svc',SVC(probability=True,random_state=42))
],
    final_estimator=RandomForestClassifier(random_state=42),
    cv=5)
stacking_clf.fit(x_train,y_train)
stacking_clf.score(x_test,y_test)

0.912

In [467]:
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5  # number of cross-validation folds
)
stacking_clf.fit(x_train, y_train)
stacking_clf.score(x_test, y_test)

0.928

In [486]:
from sklearn.datasets import fetch_openml
mnil=fetch_openml('mnist_784',as_frame=False)

/home/we/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [497]:
np.random.seed(42)
X=mnil.data
Y=mnil.target

x,x_test,y,y_test=train_test_split(X,Y,test_size=10_000)
x_train,x_cv,y_train,y_cv=train_test_split(x,y,test_size=10_000)

In [698]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

svc=LinearSVC(tol=20,random_state=42,)
trees=ExtraTreesClassifier(random_state=42)
forest=RandomForestClassifier(random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [634]:
estimators=[svc,trees,forest,mlp_clf]
for est in estimators:
    est.fit(x_train,y_train)
    print("training: ",est,' ',est.score(x_cv,y_cv))

training:  LinearSVC(random_state=42, tol=20)   0.8733
training:  ExtraTreesClassifier(random_state=42)   0.9718
training:  RandomForestClassifier(random_state=42)   0.9703
training:  MLPClassifier(random_state=42)   0.9664


In [670]:
from sklearn.ensemble import VotingClassifier

vot=VotingClassifier(
    estimators=[
        ('svc',svc),
        ('trees',trees),
        ('forest',forest)
    ]
)
vot.fit(x_train,y_train)

VotingClassifier(estimators=[('svc', LinearSVC(random_state=42, tol=20)),
                             ('trees', ExtraTreesClassifier(random_state=42)),
                             ('forest',
                              RandomForestClassifier(random_state=42))])

In [671]:
vot.score(x_cv,y_cv)

0.9695

In [672]:
y_val=np.int64(y_cv)
[est.score(x_cv,y_val) for est in vot.estimators_]

[0.8733, 0.9718, 0.9703]

In [673]:
vot.set_params(svc='drop')
svm_train=vot.named_estimators_.pop('svc')
vot.estimators_.remove(svm_train)


VotingClassifier(estimators=[('svc', 'drop'),
                             ('trees', ExtraTreesClassifier(random_state=42)),
                             ('forest',
                              RandomForestClassifier(random_state=42))])

In [712]:
vot.score(x_cv,y_cv)

0.9704

In [714]:
vot.voting='soft'
vot.score(x_cv,y_cv)

0.972

In [740]:
pred=np.empty([len(x_cv),2])
for ind,est in enumerate(vot.estimators_):
    pred[:,ind]=est.predict(x_cv)


In [742]:
blender=RandomForestClassifier()
blender.fit(pred,y_cv)
blender.score(pred,y_cv)

0.9749

In [744]:
pred=np.empty([len(x_test),2])
for ind,est in enumerate(vot.estimators_):
    pred[:,ind]=est.predict(x_test)
blender.score(pred,y_test)

0.9667

In [734]:
stack=StackingClassifier(
    estimators=[
        ('trees',trees),
        ('forest',forest)
    ],final_estimator=blender
)
stack.fit(x_train,y_train)
stack.score(x_test,y_test)

0.9721